## Rerun the Evaluation

In [1]:
import pandas as pd

ctgan_synthetic_data = []
import os
for dirname, _, filenames in os.walk("../thesisgan/output/ctabgan_hpo_new/"):
    for filename in filenames:
        if 'syn' in filename:
            fx = os.path.join(dirname, filename)
            ctgan_synthetic_data.append(fx)            
            print(fx)

../thesisgan/output/ctabgan_hpo_new/6/syn.csv
../thesisgan/output/ctabgan_hpo_new/3/syn.csv
../thesisgan/output/ctabgan_hpo_new/0/syn.csv
../thesisgan/output/ctabgan_hpo_new/8/syn.csv
../thesisgan/output/ctabgan_hpo_new/1/syn.csv
../thesisgan/output/ctabgan_hpo_new/7/syn.csv
../thesisgan/output/ctabgan_hpo_new/2/syn.csv
../thesisgan/output/ctabgan_hpo_new/5/syn.csv
../thesisgan/output/ctabgan_hpo_new/4/syn.csv
../thesisgan/output/ctabgan_hpo_new/9/syn.csv


In [2]:
real_data = pd.read_csv("../thesisgan/input/new_hpo_data.csv")
test_data = pd.read_csv("../thesisgan/input/new_test_data.csv")
import sys
sys.path.append("..")
from ctabganplus.model.evaluation import get_utility_metrics

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
syn = pd.read_csv("../thesisgan/output/ctabgan_sweep_run/9/syn.csv")

In [9]:
le_dict = {"attack_type": "le_attack_type", "label": "le_label", "proto": "le_proto", "tos": "le_tos"}
for c in le_dict.keys():
    le_dict[c] = LabelEncoder()
    test_data[c] = le_dict[c].fit_transform(test_data[c])
    real_data[c] = le_dict[c].fit_transform(real_data[c])

In [10]:
for synthetic_data in ctgan_synthetic_data:
    df = pd.read_csv(synthetic_data)
    for c in le_dict.keys():
        df[c] = le_dict[c].fit_transform(df[c])
    #if the synthetic data has only one unique value for the attack type, add a row with the attack type
    sample_data_value_counts = df["attack_type"].value_counts()
    for i in range(len(sample_data_value_counts)):
        if sample_data_value_counts[i] == 1:
            at = sample_data_value_counts.index[i]
            df = pd.concat([df,real_data[real_data["attack_type"] == at].sample(3)], ignore_index=True)
    for at in test_data["attack_type"].unique():
        if at not in df["attack_type"].unique():
        #add a row with the attack type
            df = pd.concat([df,real_data[real_data["attack_type"] == at].sample(3)], ignore_index=True)

    result_df, cr = get_utility_metrics(real_data, test_data, df, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})
    result_df["id"] = synthetic_data.split("\\")[4]
    cr["id"] = synthetic_data.split("\\")[4]
    path = synthetic_data.split("syn")[0]
    print(path)
    result_df.to_csv(path + "new_classification_results.csv", index=False)
    cr.to_csv(path + "new_classification_report.csv", index=False)
    print("Done for " + synthetic_data)

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data
..\thesisgan\output\ctabgan_sweep_run_1\0\
Done for ..\thesisgan\output\ctabgan_sweep_run_1\0\syn.csv
Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data
..\thesisgan\output\ctabgan_sweep_run_1\1\
Done for ..\thesisgan\output\ctabgan_sweep_run_1\1\syn.csv
Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained

# HPO Results Evaluation

In [3]:
import pandas as pd

ctabgan_classification_results = []
ctabgan_classification_report = []
import os
for dirname, _, filenames in os.walk("../thesisgan/output/ctabgan_hpo_new/"):
    for filename in filenames:
        if "Classification Report" in filename:
            fx = os.path.join(dirname, filename)
            ctabgan_classification_report.append(fx)            
            print(fx)
        elif "Classification Results" in filename:
            fx = os.path.join(dirname, filename)
            ctabgan_classification_results.append(fx)
            print(fx)

../thesisgan/output/ctabgan_hpo_new/6/table/Classification Results_105_d71868d3a559e723af1a.table.json
../thesisgan/output/ctabgan_hpo_new/6/table/Classification Report_105_860138129ff9065a987e.table.json
../thesisgan/output/ctabgan_hpo_new/3/table/Classification Results_105_47294caee0c6028a9ddd.table.json
../thesisgan/output/ctabgan_hpo_new/3/table/Classification Report_105_4ac41611dd1457e054c9.table.json
../thesisgan/output/ctabgan_hpo_new/0/table/Classification Results_105_a7daec6052bbd7a8bf80.table.json
../thesisgan/output/ctabgan_hpo_new/0/table/Classification Report_105_fcc073b3c41fc86ee5ff.table.json
../thesisgan/output/ctabgan_hpo_new/8/table/Classification Report_105_9750ed9a00fe82d352e9.table.json
../thesisgan/output/ctabgan_hpo_new/8/table/Classification Results_105_01c0f9b3f3c2276259ca.table.json
../thesisgan/output/ctabgan_hpo_new/1/table/Classification Report_105_4425ba1ddb0e19fff2fe.table.json
../thesisgan/output/ctabgan_hpo_new/1/table/Classification Results_105_33f7458

In [4]:
import json

In [19]:
# read every json file into a dataframe and append to a main dataframe
# before appending, add the id j0cor82r from the path of the file offline-run-20240509_081254-j0cor82r for each
ctabgan_classification_report_df = pd.DataFrame()
for file in ctabgan_classification_report:
    cr_df = json.load(open(file))
    cr_df = pd.DataFrame(cr_df["data"], columns=cr_df["columns"])
    cr_df["id"] = file.split("/")[4].split("_")[0]
    ctabgan_classification_report_df = pd.concat([ctabgan_classification_report_df, cr_df])
    ctabgan_classification_report_df.reset_index(drop=True, inplace=True)

In [20]:
ctabgan_classification_report_df

,attack_type,precision,recall,f1-score,support,Model,type,id
0,benign,1.00,1.00,1.00,261660,xgb,real,6
1,bruteForce,0.00,0.00,0.00,253,xgb,real,6
2,dos,0.95,0.99,0.97,2630,xgb,real,6
3,pingScan,0.40,0.57,0.47,7,xgb,real,6
4,portScan,1.00,1.00,1.00,62741,xgb,real,6
...,...,...,...,...,...,...,...,...
1195,pingScan,0.00,0.00,0.00,0,mlp,diff,9
1196,portScan,1.00,1.00,1.00,0,mlp,diff,9
1197,accuracy,0.00,0.00,0.19,0,mlp,diff,9
1198,macro,0.20,0.20,0.19,0,mlp,diff,9


In [21]:
pd.set_option("display.max_rows", None)

### Classification Results

In [22]:
ctabgan_classification_results_df = pd.DataFrame()
for file in ctabgan_classification_results:
    cr_df = json.load(open(file))
    cr_df = pd.DataFrame(cr_df["data"], columns=cr_df["columns"])
    cr_df["id"] = file.split("/")[4].split("_")[0]
    ctabgan_classification_results_df = pd.concat([ctabgan_classification_results_df, cr_df])
    ctabgan_classification_results_df.reset_index(drop=True, inplace=True)
    

In [23]:
ctabgan_classification_results_df

,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1,Model,Type,id
0,71.271380,0.940386,0.687912,0.079095,0.000414,0.000810,xgb,Real,6
1,34.456011,0.675576,0.169815,0.083068,0.000818,0.000656,lr,Real,6
2,71.279469,0.820541,0.688037,0.079088,0.000671,0.000810,dt,Real,6
3,71.363910,0.952038,0.688540,0.079019,0.000374,0.000809,rf,Real,6
4,59.994900,0.859328,0.592595,0.085634,0.000608,0.000859,mlp,Real,6
5,39.125930,0.712570,0.388022,0.085306,0.000791,0.000852,xgb,Fake,6
6,39.406903,0.658878,0.211198,0.085414,0.000829,0.000713,lr,Fake,6
7,44.858288,0.655364,0.296685,0.086935,0.000831,0.000798,dt,Fake,6
8,41.533949,0.648787,0.392850,0.086136,0.000834,0.000854,rf,Fake,6
9,35.957632,0.642407,0.329496,0.083881,0.000838,0.000822,mlp,Fake,6


In [93]:
ctabgan_classification_results_df[(ctabgan_classification_results_df["Type"] == "Fake") & (ctabgan_classification_results_df["Model"] != "lr")].sort_values(by="F1_Score", ascending=True)


,Acc,AUC,F1_Score,Model,Type,id,run
217,3.506696,0.396917,0.055228,dt,Fake,6,1
215,29.526456,0.611488,0.070655,xgb,Fake,6,1
260,10.681783,0.480209,0.123675,xgb,Fake,9,1
219,26.131177,0.565900,0.167571,mlp,Fake,6,1
262,19.817858,0.498862,0.179543,dt,Fake,9,1
263,24.289796,0.602864,0.181881,rf,Fake,9,1
97,39.189718,0.619936,0.188395,dt,Fake,6,1
99,44.008194,0.681676,0.201758,mlp,Fake,6,1
264,38.318576,0.650401,0.216604,mlp,Fake,9,1
22,28.429452,0.552684,0.297031,dt,Fake,1,1


In [25]:
ctabgan_classification_results_df.drop(["Model"],axis=1).groupby(["Type","id"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


Acc       AUC  F1_Score    SE_Acc    SE_AUC     SE_F1
Type       id                                                             
Real       4   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           5   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           6   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           7   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           9   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           8   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           2   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           3   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           1   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
           0   61.673134  0.849574  0.565380  0.081181  0.000577  0.000789
Fake       4   43.909212  0.659381  0.393597  0.086224  0.000826  0.000840
           2   42.145563  0.683862  0.393278  0.086091  0.000810  0.000854
           0   40.596064  0.620662  0.350050  0.085749  0.000846  0.000824
           6   40.176540  0.663601  0.323650  0.085535  0.000825  0.000808
           1   47.202498  0.678903  0.321537  0.086535  0.000815  0.000804
           3   35.383266  0.621818  0.319336  0.081575  0.000842  0.000803
Difference 7   27.702773  0.197925  0.317842  0.000908 -0.000240  0.000042
Fake       9   40.343329  0.704501  0.309072  0.085438  0.000789  0.000796
           5   34.950305  0.617619  0.289548  0.082857  0.000849  0.000775
           8   41.706662  0.668355  0.285703  0.085049  0.000814  0.000783
Difference 8   19.966472  0.181219  0.279677 -0.003869 -0.000237  0.000006
           5   26.722829  0.231954  0.275832 -0.001676 -0.000272  0.000014
           9   21.329805  0.145073  0.256307 -0.004258 -0.000213 -0.000007
Fake       7   33.970361  0.651649  0.247538  0.080273  0.000817  0.000747
Difference 3   26.289868  0.227755  0.246044 -0.000394 -0.000265 -0.000014
           1   14.470636  0.170671  0.243843 -0.005354 -0.000238 -0.000015
           6   21.496594  0.185972  0.241729 -0.004354 -0.000248 -0.000019
           0   21.077070  0.228911  0.215329 -0.004569 -0.000269 -0.000035
           2   19.527571  0.165711  0.172102 -0.004910 -0.000233 -0.000065
           4   17.763922  0.190192  0.171783 -0.005043 -0.000249 -0.000051

# HPO Results Evaluation

In [ ]:
import pandas as pd

ctgan_classification_report = []
ctgan_classification_results = []
import os
for dirname, _, filenames in os.walk("..\\thesisgan\\output\\CTGAN_HPO\\"):
    for filename in filenames:
        if "new_classification_report" in filename:
            fx = os.path.join(dirname, filename)
            ctgan_classification_report.append(fx)            
            print(fx)
        elif "new_classification_results" in filename:
            fx = os.path.join(dirname, filename)
            ctgan_classification_results.append(fx)
            print(fx)

..\thesisgan\output\CTGAN_HPO\0\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\0\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\1\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\1\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\10\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\10\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\11\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\11\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\12\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\12\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\13\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\13\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\14\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\14\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\15\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\15\new_classification_

In [ ]:
import json

In [ ]:
# read every json file into a dataframe and append to a main dataframe
# before appending, add the id j0cor82r from the path of the file offline-run-20240509_081254-j0cor82r for each

ctgan_classification_report_df = pd.DataFrame()
for file in ctgan_classification_report:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    ctgan_classification_report_df = pd.concat([ctgan_classification_report_df, cr_df])
    ctgan_classification_report_df.reset_index(drop=True, inplace=True)

In [ ]:
pd.set_option("display.max_rows", None)

### Classification Results

In [ ]:
ctgan_classification_results_df = pd.DataFrame()
for file in ctgan_classification_results:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    ctgan_classification_results_df = pd.concat([ctgan_classification_results_df, cr_df])
    ctgan_classification_results_df.reset_index(drop=True, inplace=True)

In [ ]:
ctgan_classification_results_df[ctgan_classification_results_df["Type"] == "Fake"].sort_values(by="F1_Score", ascending=False).sort_values(by="id", ascending=False).head(100)

,Acc,AUC,F1_Score,Model,Type,id
234,73.168639,0.888510,0.676596,rf,Fake,9
233,57.923670,0.737023,0.389336,dt,Fake,9
232,64.025036,0.836684,0.589617,lr,Fake,9
235,71.790143,0.924916,0.686023,mlp,Fake,9
221,33.358948,0.583493,0.278108,dt,Fake,8
222,49.351912,0.838198,0.505383,rf,Fake,8
223,49.215455,0.857012,0.523632,mlp,Fake,8
220,40.383374,0.760958,0.405967,lr,Fake,8
209,35.258814,0.595368,0.301022,dt,Fake,7
208,50.006385,0.697204,0.525267,lr,Fake,7


In [ ]:
ctgan_classification_results_df.drop(["Model"],axis=1).groupby(["id", "Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


,,Acc,AUC,F1_Score
id,Type,,,
9,Real,62.932143,0.855187,0.629673
14,Real,62.932143,0.855187,0.629673
15,Real,62.932143,0.855187,0.629673
13,Real,62.932143,0.855187,0.629673
6,Real,62.932143,0.855187,0.629673
16,Real,62.932143,0.855187,0.629673
12,Real,62.932143,0.855187,0.629673
17,Real,62.932143,0.855187,0.629673
8,Real,62.932143,0.855187,0.629673


# Evaluate Old Best Model

In [1]:
import pandas as pd
import numpy as np
from model.evaluation import get_utility_metrics, stat_sim


In [7]:
train_data = pd.read_csv("../thesisgan/input/new_train_data.csv")
test_data = pd.read_csv("../thesisgan/input/new_hpo_data.csv")
syn_data = pd.read_csv("../thesisgan/output/ctabgan_old_best_main/syn.csv")


In [8]:
attack_type_le = {"benign": 0, "bruteForce": 1, "portScan": 2, "pingScan": 3, "dos": 4}
proto_le = {"TCP": 0, "UDP": 1, "ICMP": 2, "IGMP": 3}
label_type_le = {"normal": 0, "attack": 1, "attacker": 1, "victim": 2}
tos_le = {0 : 0, 32 : 1, 192 : 2, 16 : 3}
#based on the unique values in the dataset, we will create a dictionary to map the values to integers
datasets = [train_data, test_data]
for dataset in datasets:
    dataset["attack_type"] = dataset["attack_type"].map(attack_type_le)
    dataset["proto"] = dataset["proto"].map(proto_le)
    dataset["tos"] = dataset["tos"].map(tos_le)
    dataset["label"] = dataset["label"].map(label_type_le)

In [9]:
train_data.attack_type.value_counts(), test_data.attack_type.value_counts(), syn_data.attack_type.value_counts()

(attack_type
 0    932447
 4    218536
 2     12529
 1       667
 3       153
 Name: count, dtype: int64,
 attack_type
 0    253721
 4     64172
 2      1983
 1       169
 3         5
 Name: count, dtype: int64,
 attack_type
 0    922688
 3    234539
 2      6400
 4       639
 1        66
 Name: count, dtype: int64)

In [10]:
train_data.label.value_counts(), test_data.label.value_counts(), syn_data.label.value_counts()

(label
 0    932447
 1    118252
 2    113633
 Name: count, dtype: int64,
 label
 0    253721
 1     33307
 2     33022
 Name: count, dtype: int64,
 label
 0    922473
 1    136537
 2    105322
 Name: count, dtype: int64)

In [11]:
old_best_results, old_best_cr = get_utility_metrics(train_data, test_data,
                                                                        syn_data, scaler="MinMax",
                                                                        type={"Classification":["xgb","lr","dt","rf","mlp"]})

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data


/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data


/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

Model:  rf trained on fake data


/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model:  mlp trained on fake data


/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sangtani/.conda/envs/ctabgan_new/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
old_best_results.drop(["Model"],axis=1).groupby(["Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1
Type,,,,,,
Real,78.131914,0.905732,0.606915,0.069406,0.000493,0.000842
Fake,37.902916,0.687795,0.333721,0.084990,0.000815,0.000827
Difference,40.228998,0.217936,0.273194,-0.015584,-0.000322,0.000015
